In [1]:
import glob
import os
from os.path import join

import librosa
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split
from tqdm import tqdm

In [13]:
def get_features_filtered_by_label(features, labels, selected_label):
    return [feature for feature, label in zip(features, labels) if label == selected_label]

def get_k_main_eigenvectors(features, labels, selected_label, k=1):
    selected_features = np.concatenate(get_features_filtered_by_label(features, labels, selected_label))
    _, _, vh = np.linalg.svd(normalize(selected_features), full_matrices=False)
    return vh[:k]

def get_name(path):
    return os.path.splitext(os.path.split(path)[1])[0]

def normalize(x):
    return x - x.mean()

In [3]:
paths = sorted(glob.glob('datasets/khanty_4/*.wav'))

# First variant: 
1. split train/test for whole files (not single vectors)
2. calculate SVD for both train and test

In [19]:
mfcc_list = []
labels_list = []
for path in tqdm(paths):
    y, sr = librosa.load(path)
    mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=26).T
    mfcc_list.append(mfcc)
    labels_list.append(get_name(path)[0])

100%|██████████| 520/520 [02:21<00:00,  3.67it/s]


In [20]:
X_train, X_test, y_train, y_test = train_test_split(mfcc_list, labels_list,
                                                    test_size=0.33, random_state=42, stratify=labels_list)

In [21]:
features = np.concatenate(get_features_filtered_by_label(X_train, y_train, 'F'))
_, _, vh_female = np.linalg.svd(normalize(features), full_matrices=False)
eigen_female = vh_female[0]

features = np.concatenate(get_features_filtered_by_label(X_train, y_train, 'M'))
_, _, vh_male = np.linalg.svd(normalize(features), full_matrices=False)
eigen_male = vh_male[0]

In [25]:
counter = 0
features_test = get_features_filtered_by_label(X_test, y_test, 'F')
for feature in features_test:
    _, _, vh_test = np.linalg.svd(normalize(feature), full_matrices=False)
    eigen_test = vh_test[0]
    dist_to_female = np.linalg.norm(eigen_test - eigen_female, ord=2)
    dist_to_male = np.linalg.norm(eigen_test - eigen_male, ord=2)
    correct = dist_to_female < dist_to_male
#     print(f'{dist_to_female:.3e}, {dist_to_male:.3e}, {correct}')
    if correct:
        counter += 1
print(f'{counter/len(features_test):.3f}')

0.765


In [26]:
counter = 0
features_test = get_features_filtered_by_label(X_test, y_test, 'M')
for feature in features_test:
    _, _, vh_test = np.linalg.svd(normalize(feature), full_matrices=False)
    eigen_test = vh_test[0]
    dist_to_female = np.linalg.norm(eigen_test - eigen_female, ord=2)
    dist_to_male = np.linalg.norm(eigen_test - eigen_male, ord=2)
    correct = dist_to_female > dist_to_male
#     print(f'{dist_to_female:.3e}, {dist_to_male:.3e}, {correct}')
    if correct:
        counter += 1
print(f'{counter/len(features_test):.3f}')

0.457
